![piggy_bank](piggy_bank.jpg)

<br>

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean and store the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to set up a PostgreSQL database to store this campaign's data, designing the schema in a way that would allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split, in order to save separate files based on the tables you will create. It is recommended to use `pandas` for these tasks.

Lastly, you will write the SQL code that the bank can execute to create the tables and populate with the data from the csv files. As the bank are quite strict about their security, you'll save SQL files as multiline string variables that they can then use to create the database on their end. 

You have been asked to design a database that will have three tables:

## client

| column | data type | description | original column in dataset |
|--------|-----------|-------------|----------------------------|
| `id` | `serial` | Client ID - primary key | `client_id` |
| `age` | `integer` | Client's age in years | `age` |
| `job` | `text` | Client's type of job | `job` |
| `marital` | `text` | Client's marital status | `marital` | 
| `education` | `text` | Client's level of education | `education` |
| `credit_default` | `boolean` | Whether the client's credit is in default | `credit_default` |
| `housing` | `boolean` | Whether the client has an existing housing loan (mortgage) | `housing` | 
| `loan` | `boolean` | Whether the client has an existing personal loan | `loan` |

<br>

## campaign

| column | data type | description | original column in dataset |
|--------|-----------|-------------|----------------------------|
| `campaign_id` | `serial` | Campaign ID - primary key | N/A - new column |
| `client_id` | `serial` | Client ID - references `id` in the `client` table | `client_id` |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | `campaign` |
| `contact_duration` | `integer` | Last contact duration in seconds | `duration` |
| `pdays` | `integer` | Number of days since contact in previous campaign (`999` = not previously contacted) | `pdays` |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | `previous` |
| `previous_outcome` | `boolean` | Outcome of the previous campaign | `poutcome` |
| `campaign_outcome` | `boolean` | Outcome of the current campaign | `y` |
| `last_contact_date` | `date` | Last date the client was contacted | A combination of `day`, `month`, and the newly created `year` |

<br>

## economics

| column | data type | description | original column in dataset |
|--------|-----------|-------------|----------------------------|
| `client_id` | `serial` | Client ID - references `id` in the `client` table | `client_id` |
| `emp_var_rate` | `float` | Employment variation rate (quarterly indicator) | `emp_var_rate` |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | `cons_price_idx` |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three month rate (daily indicator) | `euribor3m` |
| `number_employed` | `float` | Number of employees (quarterly indicator)| `nr_employed` |

In [89]:
import pandas as pd
import numpy as np

# Start coding here...
df = pd.read_csv('bank_marketing.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 22 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   client_id       41188 non-null  int64  
 1   age             41188 non-null  int64  
 2   job             41188 non-null  object 
 3   marital         41188 non-null  object 
 4   education       41188 non-null  object 
 5   credit_default  41188 non-null  object 
 6   housing         41188 non-null  object 
 7   loan            41188 non-null  object 
 8   contact         41188 non-null  object 
 9   month           41188 non-null  object 
 10  day             41188 non-null  int64  
 11  duration        41188 non-null  int64  
 12  campaign        41188 non-null  int64  
 13  pdays           41188 non-null  int64  
 14  previous        41188 non-null  int64  
 15  poutcome        41188 non-null  object 
 16  emp_var_rate    41188 non-null  float64
 17  cons_price_idx  41188 non-null 

In [90]:
df.columns

Index(['client_id', 'age', 'job', 'marital', 'education', 'credit_default',
       'housing', 'loan', 'contact', 'month', 'day', 'duration', 'campaign',
       'pdays', 'previous', 'poutcome', 'emp_var_rate', 'cons_price_idx',
       'cons_conf_idx', 'euribor3m', 'nr_employed', 'y'],
      dtype='object')

**Create `client_df`, `campaign_df` and `economics_df` the data frames from main data frame**

In [91]:
client_df = df[['client_id', 'age', 'job', 'marital', 'education', 'credit_default', 'housing', 'loan']]

campaign_df = df[['client_id', 'campaign',  'duration','contact', 'pdays', 'previous', 'poutcome', 'y', 'month', 'day']]

economics_df = df[['client_id', 'emp_var_rate', 'cons_price_idx', 'euribor3m', 'nr_employed']]

In [92]:
print(client_df.head())

print(campaign_df.head())

print(economics_df.head())

   client_id  age        job  marital    education credit_default housing loan
0          0   56  housemaid  married     basic.4y             no      no   no
1          1   57   services  married  high.school        unknown      no   no
2          2   37   services  married  high.school             no     yes   no
3          3   40     admin.  married     basic.6y             no      no   no
4          4   56   services  married  high.school             no      no  yes
   client_id  campaign  duration    contact  ...     poutcome   y month day
0          0         1       261  telephone  ...  nonexistent  no   may  13
1          1         1       149  telephone  ...  nonexistent  no   may  19
2          2         1       226  telephone  ...  nonexistent  no   may  23
3          3         1       151  telephone  ...  nonexistent  no   may  27
4          4         1       307  telephone  ...  nonexistent  no   may   3

[5 rows x 10 columns]
   client_id  emp_var_rate  cons_price_idx  eur

**Rename columns name for `client_df`, `campaign_df` and `economics_df` data frames**

In [93]:
client_df.rename(columns = {"client_id": "id"}, inplace = True)

campaign_df.rename(columns = {"duration": "contact_duration", "previous":"previous_campaign_contacts", "y": "campaign_outcome", "poutcome": "previous_outcome", "campaign": "number_contacts"}, inplace = True)

economics_df.rename(columns = {"euribor3m": "euribor_three_months", "nr_employed": "number_employed"}, inplace = True)

**Changing undesired paterns in `education` column value**

In [94]:
import numpy as np

client_df["education"] = client_df["education"].str.replace(".", "_").replace("unknown", np.nan)
client_df.job = client_df.job.str.replace(".", "")
client_df.head()

,id,age,job,marital,education,credit_default,housing,loan
0,0,56,housemaid,married,basic_4y,no,no,no
1,1,57,services,married,high_school,unknown,no,no
2,2,37,services,married,high_school,no,yes,no
3,3,40,admin,married,basic_6y,no,no,no
4,4,56,services,married,high_school,no,no,yes


**Check unique values in `job` column after change**

In [95]:
client_df.job.unique()

array(['housemaid', 'services', 'admin', 'blue-collar', 'technician',
       'retired', 'management', 'unemployed', 'self-employed', 'unknown',
       'entrepreneur', 'student'], dtype=object)

**Making more cleanup in `campaign_df` Data Frame**

In [96]:
import numpy as np

campaign_df["previous_outcome"] = campaign_df["previous_outcome"].replace({"success" :1, "failure": 0, "nonexistent": np.nan})

campaign_df["campaign_outcome"] = campaign_df["campaign_outcome"].replace({"yes" :1, "no": 0})

campaign_df.insert(0, "campaign_id", 1)

campaign_df.head()

,campaign_id,client_id,number_contacts,contact_duration,contact,pdays,previous_campaign_contacts,previous_outcome,campaign_outcome,month,day
0,1,0,1,261,telephone,999,0,NaN,0,may,13
1,1,1,1,149,telephone,999,0,NaN,0,may,19
2,1,2,1,226,telephone,999,0,NaN,0,may,23
3,1,3,1,151,telephone,999,0,NaN,0,may,27
4,1,4,1,307,telephone,999,0,NaN,0,may,3


**Create `last_contact_date` column and remove `month` and `day` columns from `campaign` Data Frame**

In [97]:
campaign_df["last_contact_date"] = pd.to_datetime("2022-" + campaign_df["month"].astype(str) + "-" + campaign_df["day"].astype(str))
campaign_df.drop(columns = ["month", "day"], inplace= True)
campaign_df.head()

,campaign_id,client_id,number_contacts,contact_duration,contact,pdays,previous_campaign_contacts,previous_outcome,campaign_outcome,last_contact_date
0,1,0,1,261,telephone,999,0,NaN,0,2022-05-13
1,1,1,1,149,telephone,999,0,NaN,0,2022-05-19
2,1,2,1,226,telephone,999,0,NaN,0,2022-05-23
3,1,3,1,151,telephone,999,0,NaN,0,2022-05-27
4,1,4,1,307,telephone,999,0,NaN,0,2022-05-03


**Remove All duplicates values in all Data Frames**

In [98]:
for db in [client_df, campaign_df, economics_df]:
    db.drop_duplicates(inplace = True)

**Save each Data Frame to csv file**

In [99]:
client_df.to_csv("client.csv")
campaign_df.to_csv("campaign.csv")
economics_df.to_csv("economics.csv")

**Creating Database Tables From csv files**

In [100]:
client_table = """
Create Table client (
id Serial Primary Key,
age Integer,
job Text,
marital Text,
education Text,
credit_default boolean,
housing boolean,
loan boolean
);
\copy client from 'client.csv' DELIMITER ',' CSV HEADER
"""

campaign_table = """
Create Table campaign (
campaign_id Serial Primary Key,
client_id SERIAL references client (id), 
number_contacts Integer,
contact_duration Integer,
pdays Integer,
previous_campaign_contacts Integer,
previous_outcome boolean,
campaign_outcome boolean,
last_contact_date date
);
\copy campaign from 'campaign.csv' DELIMITER ',' CSV HEADER
"""
economics_table = """
Create Table economics (
client_id SERIAL references client (id), 
emp_var_rate Float,
cons_price_idx Float,
euribor_three_months Float,
number_employed Float,
);
\copy economics from 'economics.csv' DELIMITER ',' CSV HEADER
"""

**Explore DataFrames with SQL code**

In [101]:
Select *
From client_df

,id,age,job,marital,education,credit_default,housing,loan
0,0,56,housemaid,married,basic_4y,no,no,no
1,1,57,services,married,high_school,unknown,no,no
2,2,37,services,married,high_school,no,yes,no
3,3,40,admin,married,basic_6y,no,no,no
4,4,56,services,married,high_school,no,no,yes
...,...,...,...,...,...,...,...,...
41183,41183,73,retired,married,professional_course,no,yes,no
41184,41184,46,blue-collar,married,professional_course,no,no,no
41185,41185,56,retired,married,university_degree,no,yes,no
41186,41186,44,technician,married,professional_course,no,no,no


In [102]:
Select *
From campaign_df

,campaign_id,client_id,number_contacts,contact_duration,contact,pdays,previous_campaign_contacts,previous_outcome,campaign_outcome,last_contact_date
0,1,0,1,261,telephone,999,0,NaN,0,2022-05-13 00:00:00+00:00
1,1,1,1,149,telephone,999,0,NaN,0,2022-05-19 00:00:00+00:00
2,1,2,1,226,telephone,999,0,NaN,0,2022-05-23 00:00:00+00:00
3,1,3,1,151,telephone,999,0,NaN,0,2022-05-27 00:00:00+00:00
4,1,4,1,307,telephone,999,0,NaN,0,2022-05-03 00:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...
41183,1,41183,1,334,cellular,999,0,NaN,1,2022-11-30 00:00:00+00:00
41184,1,41184,1,383,cellular,999,0,NaN,0,2022-11-06 00:00:00+00:00
41185,1,41185,2,189,cellular,999,0,NaN,0,2022-11-24 00:00:00+00:00
41186,1,41186,1,442,cellular,999,0,NaN,1,2022-11-17 00:00:00+00:00


In [105]:
Select *
From economics_df

,client_id,emp_var_rate,cons_price_idx,euribor_three_months,number_employed
0,0,1.1,93.994,4.857,5191.0
1,1,1.1,93.994,4.857,5191.0
2,2,1.1,93.994,4.857,5191.0
3,3,1.1,93.994,4.857,5191.0
4,4,1.1,93.994,4.857,5191.0
...,...,...,...,...,...
41183,41183,-1.1,94.767,1.028,4963.6
41184,41184,-1.1,94.767,1.028,4963.6
41185,41185,-1.1,94.767,1.028,4963.6
41186,41186,-1.1,94.767,1.028,4963.6
